# Movie Ranker

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = (SparkSession.builder
    .master('local')
    .appName('myAppName')
    .getOrCreate())

spark.sparkContext.setLogLevel("ERROR")

## Read the parquet file generated with pandas and omdb api

In [ ]:
df = spark.read.parquet('data/movies.parquet.gzip')
print(f'{df.count()} movies loaded')

## filter the movies with a rating less than 6.0

In [ ]:
bad_movies = df.filter(col('imdbrating') < 6.0)
bad_movies_count = bad_movies.count()
print(f'We have {bad_movies_count} movies with a ranking less than 6.0')

In [ ]:
bad_movies.sort(col('imdbrating').asc()).show(truncate=False)

In [ ]:
unrated_movies = df.where(col('imdbrating') == 'N/A')
print(f'We have {unrated_movies.count()} movies without ranking')

In [ ]:
unrated_movies.show(truncate=False)